In this notebook we'll be working with generative adversarial models (GANs).  

`make_pos` defines a starircase array of size `d`. To start, a bunch of 0-1 uniform random numbers are arranged as an `m`x`d` matrix. The code then does a running sum, also known as a cumulative sum or `cumsum`, across the rows of the matrix. `numpy` refers to moving across the rows as traversing axis 1 of the matrix, so that’s the parameter value we use in the call.

So, now we no longer have a bunch of random numbers between 0 and 1. Instead, each row contains the running sum at each location. This procedure guarantees an increasing sequence, with a starting value that is somewhere between 0 and 1 and an ending value around 10.0. It’s not a staircase yet, since a staircase goes from 0 to 1. We can rescale these values so they are in the right range. First, we grab the minimums or mins of each row, in a vector `X` . Since the rows are increasing, the min must be on the far left. We subtract off these `mins`.

We then grab the maximum, `maxs`, of each row. They are on the far right. We divide the vector `X` by the max values so everything is in the 0-1 range as desired.


In [1]:
import matplotlib.pyplot as plt

# Create a dataset of vectors
import numpy as np

# vector dimension
d = 20
# number of training examples
m = 10000

def make_pos(m,d):
  X = np.cumsum(np.random.uniform(0,1, size=(m,d)),axis=1)
  mins = X[:,0]
  X = X - mins.reshape(m,1)
  maxs = X[:,(d-1)]
  X = X / maxs.reshape(m,1)
  return(X)

Xpos = make_pos(m,d)
Xneg = make_pos(m,d)
Xneg = np.apply_along_axis(np.random.permutation, 1, Xneg)
X = np.concatenate((Xpos,Xneg))
#y = np.concatenate((np.repeat([0,1],m), np.repeat([1,0],m)))
y = np.concatenate((np.concatenate((np.repeat(1,m).reshape(m,1),np.repeat(0,m).reshape(m,1)),axis=1),
  np.concatenate((np.repeat(0,m).reshape(m,1),np.repeat(1,m).reshape(m,1)),axis=1)))

# split into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5)

Then, we’ll get a `model` to learn to produce these artificially generated, one-dimensional grayscale staircases. Staircases are a nice stand-in for the more complex 2-dimensional color images that GANs typically produce.

For our classifier, we’ll create a simple neural network using `Keras`. Our `model` has `d=20` input units to correspond to the 20 steps. There are 10 hidden units that the `model` can use to compute. And, there are 2 output units, corresponding to “staircase” or “not staircase”. 

The predictions in the output layer use a `softmax` activation. `softmax` accentuates the largest value in a vector, then normalizes the values to look like probabilities. That is, the values from `softmax` are between zero and 1 and sum up to 1.


In [2]:
# set up a keras classifier
!pip install keras=='2.3.1'
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import Model

input = Input((d,))
x = Dense(10, activation='relu')(input)
out = Dense(2, activation='softmax',name='predict')(x)

model = Model(input, out)

     |████████████████████████████████| 378kB 4.7MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


Below we compile the model.  We'll use a `mean_squared_error` `loss`, use `adam` as our `optimizer`, and record `accuracy` when printing our metrics.

In [3]:
# compile the keras model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

We train the model, looping through all the training  data `epochs = 15` times.

In [24]:
# train

model.fit(X_train, y_train, epochs=15, batch_size=10)

Epoch 1/15
1000/1000 [==============================] - 2s 2ms/step - loss: 2.1474e-05 - accuracy: 1.0000
Epoch 2/15
1000/1000 [==============================] - 2s 2ms/step - loss: 1.7496e-05 - accuracy: 1.0000
Epoch 3/15
1000/1000 [==============================] - 2s 2ms/step - loss: 1.1766e-05 - accuracy: 1.0000
Epoch 4/15
1000/1000 [==============================] - 2s 2ms/step - loss: 9.7397e-06 - accuracy: 1.0000
Epoch 5/15
1000/1000 [==============================] - 2s 2ms/step - loss: 4.1343e-05 - accuracy: 1.0000
Epoch 6/15
1000/1000 [==============================] - 2s 2ms/step - loss: 5.7074e-06 - accuracy: 1.0000
Epoch 7/15
1000/1000 [==============================] - 2s 2ms/step - loss: 4.0196e-06 - accuracy: 1.0000
Epoch 8/15
1000/1000 [==============================] - 2s 2ms/step - loss: 8.8127e-06 - accuracy: 1.0000
Epoch 9/15
1000/1000 [==============================] - 2s 2ms/step - loss: 7.6079e-06 - accuracy: 1.0000
Epoch 10/15
1000/1000 [=======================

Below we import another `keras` module we'll be using to build our models.

In [20]:
from tensorflow.keras import backend as K

Our `misordered` function counts the number of time an example in our data set is out of order realtive to its neighbors.

In [21]:
def misordered(l):
  bad = 0
  for i in range(len(l)-1):
    for j in range(i+1, len(l)):
      if l[i] > l[j]: bad += 1
  return(bad)

Below are some example images from our training set with their labels.

In [22]:
# show some examples
from keras.preprocessing import image

def make_image(v):
  v = v.reshape(1,d,1)
  v = np.concatenate((v,v,v,v,v))
  img = image.array_to_img(v)
  return(img)

for i in range(10):
  display(make_image(X_train[i]))
  print(y_train[i])

[0 1]


[0 1]


[1 0]


[1 0]


[0 1]


[1 0]


[1 0]


[1 0]


[0 1]


[0 1]


Now that we can accurately discriminate between staircase images and scrambled staircase images, let’s use the gradient approach to turn a non-staircase into a staircase.

We’ll take steps in the direction of the computed gradient until the image is recognized with over 95 percent confidence as a staircase.

We want the staircase, which is represented by the vector `input_data`, to remain a valid image. To keep the values of the vector in the right range, we reset back to zero any values in `input_data` that have gone negative.


In [23]:
import random

# get the symbolic outputs of each "key" layer (we gave them unique names).
before = 0
after = 0
for i in range(10):
  layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])

  target_class = 0

  learning_rate = 0.0001

  input_tensor = model.input

  x = layer_dict['predict'].output[:, target_class]
  objective = K.sum(x)
  grads = K.gradients(objective, input_tensor)[0]
  grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-12)
  iterate = K.function([input_tensor], [objective, grads])

  input_data = Xneg[i]

  print(input_data)

  print(misordered(input_data))

  before += misordered(input_data)
  print(input_data)
  print(model.predict(input_data.reshape(1,-1)))
  make_image(input_data)

  loss_value = 0.0
  while loss_value < 0.95: 
    loss_value, grads_value = iterate(input_data.reshape(1,-1))

    input_data += learning_rate * grads_value[0]
    input_data[input_data <0 ] = 0.0
    if random.random() < 0.01: print("**",loss_value, input_data)

  print(misordered(input_data))
  after += misordered(input_data)
  print(input_data)
  print(model.predict(input_data.reshape(1,-1)))
  make_image(input_data)

RuntimeError: ignored

Here's the constructed sequence that the discriminator thinks is a staircase.

In [17]:
input_data

array([0.9436739 , 0.92796105, 0.16642212, 0.61752079, 0.98875052,
       0.56660347, 0.64605225, 0.82673424, 0.62387285, 0.35658359,
       0.4564223 , 0.82857836, 0.07512545, 0.07262095, 0.09894495,
       0.21133531, 0.00147751, 0.23410808, 0.60519986, 0.72359434])

Below we'll learn to generate staircase images using a GAN


So, let’s try the GAN idea. It’s adversarial, so we need to construct two networks.

We define our discriminator using `build_discriminator`. The discriminator network takes as input a vector of size `d=20` representing the purported staircase. It has two fully connected or dense sets of weights leading to hidden layers, each of size `d/2` = 10. Finally, the output layer is a single unit that should output 1 for a real staircase and 0 for a fake one.

Since we know this output value is between zero and one, we can use a `sigmoid` activation function. `sigmoid` smoothly transitions from near-zero values for negative inputs to near-one values for positive inputs, which guarantees the outputs stay in a zero-to-one range.

We build the generator using `build_generator`. The generator has a similar structure to the discriminator. This network-structured generator maps a vector of `noise` into a staircase vector. The number of components of the noise vector can be thought of as a latent dimensionality of the image space --- the more noise components, the higher the latent dimensionality. We’ll use `latent_dim = 50` to be a variable that specifies the number of random values we’ll use as our input so the network would have a large pool of random values to choose from.

The generator uses a dense layer of weights to map the chosen 18 random values to a hidden layer, where the number of hidden units is 2*d = 40. The generator then has another Dense layer of size d=20, then an output layer of size `d=20`. Since all of the outputs should be between zero and one, I set the activation function for the output layer to be a sigmoid for each of these units.


In [18]:
# Let's try a GAN on it.
# from https://github.com/eriklindernoren/Keras-GAN/blob/master/dcgan/dcgan.py

from keras.optimizers import Adam

import matplotlib.pyplot as plt

def build_generator():
  noise = Input(shape=(latent_dim,))
  x = Dense(2*d, activation='relu')(noise)
  x = Dense(d, activation='relu')(x)
  out = Dense(d, activation='sigmoid')(x)
  model = Model(noise, out)
  return(model)

def build_discriminator():
  input = Input((d,))
  x = Dense(d//2, activation='relu')(input)
  x = Dense(d//2, activation='relu')(x)
  out = Dense(1, activation='sigmoid',name='predict')(x)
  model = Model(input, out)
  return(model)

def save_imgs(epoch):
  swaps = 0
  # fake
  for i in range(10):
    noise = np.random.normal(0, 1, (1, latent_dim))
    gen_img = generator.predict(noise)
    display(make_image(gen_img))
    label = discriminator.predict(gen_img)
    print(label)
    swaps += misordered(gen_img[0])
  print(swaps/10)
  # real
  for i in range(10):
    gen_img = X[i].reshape(1,d)
    display(make_image(gen_img))
    label = discriminator.predict(gen_img)
    print(label)

We’ll use the `Adam` optimizer for training, and we’ll train for `epochs=300`. The `Adam` optimizer is an adaptive algorithm introduced in 2015 that controls the process of gradient descent by optimizing the rate of learning at each step of the optimization process. Adam generally does a good job, and it quickly became a default choice for training networks.

We’ll give the networks a `batch_size` of 128 examples in each epoch.
We’ll set up our training set `X` using only positive examples of staircases. We force `X` to be the right shape for the network by calling `expand_dims` to add another dimension.

We also create targets for learning --- we use ones for valid examples of staircases from the training set `X` and zeros for fake examples that come out of the generator.
Next, we construct the combined network. We build the discriminator and set it up to be trained.

We can then instantiate a generator, `z`, which will take noise inputs and produce images, `img`.

We want to train the generator to make staircases that the discriminator cannot reliably distinguish from the real staircase images. 

 The weights of the discriminator need to remain constant while we update the generator. If we did not hold the discriminator’s weights fixed, gradient descent would "solve" the problem of fooling the discriminator merely by making the DISCRIMINATOR less accurate. So, we set `discriminator.trainable` for the combined network to `False`.

The discriminator assesses the generator's image output img and produces a judgment as to whether the image is `valid`. The combined network, end to end, connects the input` z` to the output called “valid”.

In [19]:
# Run the GAN
latent_dim = 50
optimizer = Adam(0.0002, 0.5)
epochs = 30000

# Build and compile the discriminator
discriminator = build_discriminator()

# Build the generator
generator = build_generator()

# The generator takes noise as input and generates an image
z = Input(shape=(latent_dim,))
img = generator(z)

# The discriminator takes generated images as input and determines validity
valid = discriminator(img)

# The combined model (stacked generator and discriminator)
combined = Model(z, valid)

X = Xpos
batch_size = 128
save_interval = 10

# Adversarial ground truths
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

for epoch in range(epochs):
  # Select a random half of images
  idx = np.random.randint(0, X.shape[0], batch_size)
  imgs = X[idx]

  # Sample noise and generate a batch of new images
  noise = np.random.normal(0, 1, (batch_size, latent_dim))
  gen_imgs = generator.predict(noise)

  # Train the discriminator (real classified as valid/ones and generated as fake/zeros)
  discriminator.trainable = True
  discriminator.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['accuracy'])
  for i in range(100):
    d_loss_real = discriminator.train_on_batch(imgs, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

  # Train the generator (wants discriminator to mistake images as real)
  discriminator.trainable = False
  combined.compile(loss='mean_squared_error', optimizer=optimizer)

  g_loss = combined.train_on_batch(noise, valid)

  # If at save interval => save generated image samples
  if epoch % save_interval == 0 or epoch == epochs - 1:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                save_imgs(epoch)

ValueError: ignored